In [1]:
from pyspark.sql import SparkSession
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages io.delta:delta-spark_2.12:3.1.0 pyspark-shell"
)

spark = (
    SparkSession.builder
    .appName("FeatureStore-Time-Travel")
    .master("local[*]")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog"
    )
    .getOrCreate()
)

spark


25/12/28 08:05:25 WARN Utils: Your hostname, RidhiGuntur resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/12/28 08:05:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ridhi/feature-store-delta/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ridhi/.ivy2/cache
The jars for the packages stored in: /home/ridhi/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-642e7cdf-6f59-45c3-a8f5-6f066293d76b;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 177ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

In [2]:
features_path = "/home/ridhi/feature-store-delta/delta/features/user_features"


In [3]:
features_v0 = spark.read \
    .format("delta") \
    .option("versionAsOf", 0) \
    .load(features_path)

features_v0.show()


25/12/28 08:05:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+-------+------------+------------+---------------+
|user_id|total_events|total_clicks|avg_event_value|
+-------+------------+------------+---------------+
|      1|           2|           2|            4.0|
|      3|           1|           0|            4.0|
|      2|           2|           1|            1.5|
+-------+------------+------------+---------------+



In [4]:
features_latest = spark.read \
    .format("delta") \
    .load(features_path)

features_latest.show()


+-------+------------+------------+---------------+
|user_id|total_events|total_clicks|avg_event_value|
+-------+------------+------------+---------------+
|      1|           2|           2|            5.0|
|      3|           1|           0|            5.0|
|      2|           2|           1|            2.5|
+-------+------------+------------+---------------+



In [5]:
history_df = spark.sql(
    f"DESCRIBE HISTORY delta.`{features_path}`"
)

history_df.show(truncate=False)


+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1      |2025-12-28 08:04:22.889|NULL  |NULL    |WRITE    |{mode -> Overwrite, partitionBy -> []}|NULL|NULL    |NULL     |0          |Serializable  |false        |{numFiles -> 1, nu

In [12]:
features_time = spark.read \
    .format("delta") \
    .option("timestampAsOf", "2025-12-28 08:04:22.889") \
    .load(features_path)

features_time.show()


+-------+------------+------------+---------------+
|user_id|total_events|total_clicks|avg_event_value|
+-------+------------+------------+---------------+
|      1|           2|           2|            5.0|
|      3|           1|           0|            5.0|
|      2|           2|           1|            2.5|
+-------+------------+------------+---------------+

